In [1]:
from langchain_openai import ChatOpenAI
from utils.prompts import restate_system_message, judge_system_message, response_system_message
from utils.data_utils import parse_dict_to_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults
from database import essay_database
from typing import TypedDict, Annotated, List
from langgraph.graph import START, END, StateGraph
from langgraph.constants import Send
import operator
import asyncio
from langchain_core.tools import tool
import os
from langgraph.prebuilt import ToolNode

In [2]:
class AgentState(TypedDict):
    user_query: str
    restate_query: str
    document: List[dict]
    cur_check: dict
    filtered_document: Annotated[list[dict], operator.add]

In [12]:
search = TavilySearchResults(
        max_results=5,
        search_depth="advanced")
for p in search.stream("基于双语字典的匹配去筛选SFT数据的算法名称是什么"):
    print(p)

[{'url': 'https://www.nowcoder.com/discuss/614470531504435200', 'content': '简介：该项目是由复旦大学发布的针对医疗健康对话式场景而设计的医疗领域大模型与数据集，该模型由DISC-Med-SFT数据集基于Baichuan-13B-Base指令微调得到。'}, {'url': 'https://wqw547243068.github.io/llm_train', 'content': 'L2P 算法是一种较为常用的Prompt selection算法，该算法设计了一种Key-Query的Prompt匹配方法，为每一个Prompt提供一个可学习的索引键k，即 P={(k1,P1),(k2,'}, {'url': 'https://aclanthology.org/2024.ccl-2.pdf', 'content': '... 名为ChatGPT-RetrievalQA的数据集，它基于大语言. 模型响应用户查询生成合成文档来构建。他们利用此数据集和人工生成的数据微调了一系列重. 排序器。在多个'}, {'url': 'https://github.com/Tele-AI/Telechat/blob/master/README.md', 'content': 'TeleChat的分词算法是BBPE算法，该算法是字节级实现的分词算法，任意Unicode字符都可以被表示。 TeleChat 的分词器词表大小为160256，是中英双语的词表。 BBPE算法的实现工具'}, {'url': 'https://arxiv.org/html/2411.07715v1', 'content': '语言模型的目标是通过模拟文本数据的生成概率，实现对自然语言的理解和生成。最初的模型，如n-gram，依赖于统计分析来预测词语序列，但这些模型难以捕捉复杂的'}]


In [11]:
res

[{'url': 'https://www.nowcoder.com/discuss/614470531504435200',
  'content': '简介：该项目是由复旦大学发布的针对医疗健康对话式场景而设计的医疗领域大模型与数据集，该模型由DISC-Med-SFT数据集基于Baichuan-13B-Base指令微调得到。'},
 {'url': 'https://wqw547243068.github.io/llm_train',
  'content': 'L2P 算法是一种较为常用的Prompt selection算法，该算法设计了一种Key-Query的Prompt匹配方法，为每一个Prompt提供一个可学习的索引键k，即 P={(k1,P1),(k2,'},
 {'url': 'https://aclanthology.org/2024.ccl-2.pdf',
  'content': '... 名为ChatGPT-RetrievalQA的数据集，它基于大语言. 模型响应用户查询生成合成文档来构建。他们利用此数据集和人工生成的数据微调了一系列重. 排序器。在多个'},
 {'url': 'https://github.com/Tele-AI/Telechat/blob/master/README.md',
  'content': 'TeleChat的分词算法是BBPE算法，该算法是字节级实现的分词算法，任意Unicode字符都可以被表示。 TeleChat 的分词器词表大小为160256，是中英双语的词表。 BBPE算法的实现工具'},
 {'url': 'https://arxiv.org/html/2411.07715v1',
  'content': '语言模型的目标是通过模拟文本数据的生成概率，实现对自然语言的理解和生成。最初的模型，如n-gram，依赖于统计分析来预测词语序列，但这些模型难以捕捉复杂的'}]

In [15]:
res = chain.invoke({"input": "他是个骗子"})


In [16]:
res

AIMessage(content="He's a fraud.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 31, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'qwen-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bde96db8-86cd-4c97-94a9-57c59f8cde3b-0', usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [17]:
template = ChatPromptTemplate.from_messages([
    ("human", "Hello, how are you?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "That's good to hear."),
])

In [18]:
template

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, how are you?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="I'm doing well, thanks!"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="That's good to hear."), additional_kwargs={})])

In [19]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

example_prompt = ChatPromptTemplate.from_messages(
[('human', 'What is {input}?'), 
 ('ai', '{output}')]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    # This is a prompt template used to format each individual example.
    example_prompt=example_prompt,
)

In [20]:
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'input': '2+2', 'output': '4'}, {'input': '2+3', 'output': '5'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='What is {input}?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}'), additional_kwargs={})]))

In [21]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template("Say {foo}")
prompt.format(foo="bar")

'Say bar'